captsone interpreter
https://www.capstone-engine.org/

In [2]:
%%file /tmp/test.c

int myadd(int a, int b) {
    return a + b;
}


Writing /tmp/test.c


In [3]:
! gcc -c -o /tmp/test.so /tmp/test.c

In [4]:
from capstone import *

CODE = b"\x55\x48\x8b\x05\xb8\x13\x00\x00"

md = Cs(CS_ARCH_X86, CS_MODE_64)
for i in md.disasm(CODE, 0x1000):
    print("0x%x:\t%s\t%s" %(i.address, i.mnemonic, i.op_str))


0x1000:	push	rbp
0x1001:	mov	rax, qword ptr [rip + 0x13b8]


In [ ]:
from capstone import *

CODE = b"\x55\x48\x8b\x05\xb8\x13\x00\x00"
from collections import defaultdict
class State():
    mem = defaultdict(None)
    reg = defaultdict(None)

def interp(op, state):
    
    match op.mnemonic:
        case "add":
            state.reg[op.operands[0].reg] += state[op.operands[1].reg]
        case "sub":
            state[op.operands[0].reg] -= state[op.operands[1].reg]
        case "mov":
            if op.operands[0].type == X86_OP_IMM:
                state[op.operands[0].reg] = op.operands[1].imm
            elif op.operands[0].type == X86_OP_REG:
                op.operands[0].reg = op.operands[1].reg
            elif op.operands[0].type == X86_OP_MEM:
                state[op.operands[0].reg] = mem[state[op.operands[1].reg + op.operands[1].disp]
    return state
state = defaultdict(None)
for i in md.disasm(CODE, 0x1000):
    print(dir(i))
    interp(i, state)


['_CsInsn__gen_detail', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cs', '_raw', 'address', 'bytes', 'errno', 'group', 'group_name', 'groups', 'id', 'insn_name', 'mnemonic', 'op_count', 'op_find', 'op_str', 'reg_name', 'reg_read', 'reg_write', 'regs_access', 'regs_read', 'regs_write', 'size']
['_CsInsn__gen_detail', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cs',

CsError: Details are unavailable (CS_ERR_DETAIL)

Ok so it gets hairier and harrier

Getting the smeanuitcs



# Pcode

In [ ]:
import pypcode
class State():
    space = defaultdict(defaultdict)

def interp(insn, state):
    match insn.op:
        case pypcode.PcodeOp.INT_ADD:
            


In [1]:
%%file /tmp/foo.c
int foo(int x, int y){
    return x > y ? x : y;
}

Writing /tmp/foo.c


In [2]:
%%bash
gcc /tmp/foo.c -c -o /tmp/foo.o
objdump -d -F /tmp/foo.o


/tmp/foo.o:     file format elf64-x86-64


Disassembly of section .text:

0000000000000000 <foo> (File Offset: 0x40):
   0:	f3 0f 1e fa          	endbr64 
   4:	55                   	push   %rbp
   5:	48 89 e5             	mov    %rsp,%rbp
   8:	89 7d fc             	mov    %edi,-0x4(%rbp)
   b:	89 75 f8             	mov    %esi,-0x8(%rbp)
   e:	8b 55 f8             	mov    -0x8(%rbp),%edx
  11:	8b 45 fc             	mov    -0x4(%rbp),%eax
  14:	39 c2                	cmp    %eax,%edx
  16:	0f 4d c2             	cmovge %edx,%eax
  19:	5d                   	pop    %rbp
  1a:	c3                   	ret    


In [3]:
import pypcode
# https://github.com/philzook58/pcode2c/blob/main/pcode2c/printer.py
from pypcode import Context, PcodePrettyPrinter
ctx = Context("x86:LE:64:default")
file_offset = 0x40
filename = "/tmp/foo.o"
size = 0x1a
start_address = 0
with open(filename, "rb") as file:
    file.seek(file_offset)
    code = file.read(size)
res = ctx.translate(code, base_address=start_address)
for op in res.ops:
    print(PcodePrettyPrinter.fmt_op(op))

IMARK ram[0:4]
IMARK ram[4:1]
unique[10c00:8] = RBP
RSP = RSP - 0x8
*[ram]RSP = unique[10c00:8]
IMARK ram[5:3]
RBP = RSP
IMARK ram[8:3]
unique[4400:8] = RBP + 0xfffffffffffffffc
unique[6780:4] = EDI
*[ram]unique[4400:8] = unique[6780:4]
IMARK ram[b:3]
unique[4400:8] = RBP + 0xfffffffffffffff8
unique[6780:4] = ESI
*[ram]unique[4400:8] = unique[6780:4]
IMARK ram[e:3]
unique[4400:8] = RBP + 0xfffffffffffffff8
unique[da80:4] = *[ram]unique[4400:8]
EDX = unique[da80:4]
RDX = zext(EDX)
IMARK ram[11:3]
unique[4400:8] = RBP + 0xfffffffffffffffc
unique[da80:4] = *[ram]unique[4400:8]
EAX = unique[da80:4]
RAX = zext(EAX)
IMARK ram[14:2]
unique[27c00:4] = EDX
CF = unique[27c00:4] < EAX
OF = sborrow(unique[27c00:4], EAX)
unique[27d00:4] = unique[27c00:4] - EAX
SF = unique[27d00:4] s< 0x0
ZF = unique[27d00:4] == 0x0
unique[15080:4] = unique[27d00:4] & 0xff
unique[15100:1] = popcount(unique[15080:4])
unique[15180:1] = unique[15100:1] & 0x1
PF = unique[15180:1] == 0x0
IMARK ram[16:3]
unique[eb00:1] = 

In [73]:
BV8  = BitVecSort(8)
BV16 = BitVecSort(16)
BV32 = BitVecSort(32)
BV64 = BitVecSort(64)

Select16 = Function("Select16", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(16))
Select32 = Function("Select32", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(32))
Select64 = Function("Select64", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(64))
def MSelect(a, i, n):
    if n == 1:
        return a[i]
    if n == 2:
        return Select16(a, i)
    if n == 4:
        return Select32(a, i)
    if n == 8:
        return Select64(a, i)
    else:
        assert False, "n must be 1, 2, 4 or 8"
Store16 = Function("Store16", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(16), ArraySort(BitVecSort(64), BitVecSort(8)))
Store32 = Function("Store32", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(32), ArraySort(BitVecSort(64), BitVecSort(8)))
Store64 = Function("Store64", ArraySort(BitVecSort(64), BitVecSort(8)), BitVecSort(64), BitVecSort(64), ArraySort(BitVecSort(64), BitVecSort(8)))

def MStore(a,i,v):
    size = v.size()
    if size == 1:
        return Store(a, i, v)
    if size == 2:
        return Store16(a, i, v)
    if size == 4:
        return Store32(a, i, v)
    if size == 8:
        return Store64(a, i, v)
    else:
        assert False, "size must be 1, 2, 4 or 8"

In [14]:
state[8,2]

Select(state, 8, 2)

In [7]:
from pypcode import OpCode
from z3 import *
Space = StringSort()
Varnode = Datatype('Varnode')
Varnode.declare('v', ('space', Space), ('offset', BitVecSort(64)))
Varnode = Varnode.create()

State = ArraySort(Varnode, BitVecSort(8))
#Store("constant", Lambda([x], x))
x = BitVec("x", 64)
State0 = {"ram" : None, "unique" : None, "const" : Lambda([x], x)}

def varnode(vnode, ):
    if vnode == None:
        return None
    elif vnode.space.name == "const":
        return BitVecVal(vnode.offset, vnode.size * 8)
    return BitVecVal(vnode.offset, vnode.size * 8)

def step(op, state1, state2):
    output = varnode(op.output)
    inputs = [varnode(i) for i in op.inputs]
    match op.opcode:
        case OpCode.IMARK:
            return f"IMARK "
        case OpCode.COPY:
            return MStore(state1, output, state1[inputs[0]])
        case OpCode.INT_SUB:
            return MStore(state1, output, state1[inputs[0]] - state1[inputs[1]])
        case _:
            print("unrecognized opcode", op.opcode)
            assert False
state1, state2 = Consts("state1 state2", State)
for op in res.ops:
    print(step(op, state1, state2))

IMARK 
IMARK 


Z3Exception: Sort mismatch

https://spinsel.dev/assets/2020-06-17-ghidra-brainfuck-processor-1/ghidra_docs/language_spec/html/pcoderef.html

In [77]:
from z3 import *
from collections import namedtuple
def MStore(a,i,v):
    size = v.size()
    assert size in [8,16,32,64]
    select = Function(f"select{size}", MemSort, BitVecSort(64), BitVecSort(size), MemSort)
    return select(a,i,v)
def MSelect(a, i, n):
    assert size in [8,16,32,64]
    select = Function(f"select{n}", MemSort, BitVecSort(64), BitVecSort(n))
    return select(a,i)

def varnode(vnode, state):
    if vnode == None:
        return None
    if vnode.space.name == "const":
        print(vnode.size)
        return BitVecVal(vnode.offset, vnode.size * 8)
    v = BitVecVal(vnode.offset, 64)
    if vnode.space.name == "ram":
        return MSelect(state.ram, v, vnode.size*8)
    elif vnode.space.name == "unique":
        return MSelect(state.unique, v, vnode.size*8)
    elif vnode.space.name == "register":
        return MSelect(state.register,vnode.offset, vnode.size*8)
    else:
        raise ValueError("unrecognized space", vnode.space.name)
def step(op, state1):
    control = None
    output = None
    if op.opcode == OpCode.IMARK:
        return f"IMARK"
    elif op.opcode == OpCode.BRANCH:
        control = True
        return "branch"
    elif op.opcode == OpCode.CBRANCH:
        control = inputs[0] != 0
        return "cbranch"
    elif op.opcode == OpCode.CALL:
        return "call"
    elif op.opcode == OpCode.CALLIND:
        return "callind"
    elif op.opcode == OpCode.RETURN:
        return "return"
    
    inputs = [varnode(i,state) for i in op.inputs]
    if op.opcode == OpCode.STORE:
        #assert op.inputs[0].space.name == "ram"
        return state1._replace(ram = MStore(state1.ram, inputs[1], inputs[2]))
    elif op.opcode == OpCode.LOAD:
        #assert op.inputs[0].space.name == "ram"
        return MSelect(state.ram, inputs[1], 8)
    else:
        output = arith_op(op, inputs)
    
    

def arith_op(op, inputs):
    match op.opcode:
        case OpCode.COPY:
            return inputs[0]
        case OpCode.INT_EQUAL:
            return If(inputs[0] == inputs[1], BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_NOTEQUAL:
            return If(inputs[0] != inputs[1], BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_LESS:
            return If(ULT(inputs[0], inputs[1]), BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_SLESS:
            return If(inputs[0] < inputs[1], BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_LESSEQUAL:
            return If(ULE(inputs[0], inputs[1]), BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_SLESSEQUAL:
            return If(inputs[0] <= inputs[1], BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.INT_ZEXT: 
            return ZeroExt((op.output.size - op.inputs[0].size) * 8, inputs[0])
        case OpCode.INT_SEXT:
            return SignExt((op.output.size - op.inputs[0].size) * 8, inputs[0])
        case OpCode.INT_ADD:
            return inputs[0] + inputs[1]
        case OpCode.INT_SUB:
            return inputs[0] - inputs[1]
        case OpCode.INT_CARRY:
            assert If(BVAddNoOverflow(inputs[0], inputs[1], False), BitVecVal(0, 8), BitVecVal(1, 8))
        case OpCode.INT_SCARRY:
            assert If(BVAddNoOverflow(inputs[0], inputs[1], True), BitVecVal(0, 8), BitVecVal(1, 8))
        case OpCode.INT_SBORROW:
            assert False, "unimplemented"
        case OpCode.INT_2COMP:
            return -inputs[0]
        case OpCode.INT_NEGATE:
            return ~inputs[0]
        case OpCode.INT_XOR:
            return inputs[0] ^ inputs[1]
        case OpCode.INT_AND:
            return inputs[0] & inputs[1]
        case OpCode.INT_OR:
            return inputs[0] | inputs[1]
        case OpCode.INT_LEFT:
            return inputs[0] << inputs[1]
        case OpCode.INT_RIGHT:
            return  LShR(inputs[0], inputs[1])
        case OpCode.INT_SRIGHT:
            return inputs[0] >> inputs[1]
        case OpCode.INT_MULT:
            return inputs[0] * inputs[1]
        case OpCode.INT_DIV:
            return UDiv(inputs[0], inputs[1])
        case OpCode.INT_REM:
            return URem(inputs[0], inputs[1])
        case OpCode.INT_SDIV:
            return inputs[0] / inputs[1]
        case OpCode.INT_SREM:
            return inputs[0] % inputs[1]
        case OpCode.BOOL_NEGATE:
            return If(inputs[0] == 0, BitVecVal(1, 8), BitVecVal(0, 8))       
        case OpCode.BOOL_XOR:
            return If(inputs[0] != inputs[1], BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.BOOL_AND:
            return If(inputs[0] & inputs[1] == 1, BitVecVal(1, 8), BitVecVal(0, 8))
        case OpCode.POPCOUNT:
            return BitVecVal(0, output.size*8)
            for i in range(inputs[0].size()):
                output += (inputs[0] >> i) & 1
        case _:
            raise ValueError("unrecognized opcode", op.opcode)
    return output
State = namedtuple("state", "ram unique register")
MemSort = ArraySort(BitVecSort(64), BitVecSort(8))
state = State(Const("ram", MemSort), Const("unique", MemSort), Const("register", MemSort))
for op in res.ops:
    print(PcodePrettyPrinter.fmt_op(op))
    print(step(op, state))

IMARK ram[0:4]
IMARK
IMARK ram[4:1]
IMARK
unique[10c00:8] = RBP


AssertionError: 

In [36]:
def varnode(vnode):
    if vnode == None:
        return None
    v = BitVecVal(vnode.offset, vnode.size * 8)
    if vnode.space.name == "const":
        return v
    elif vnode.space.name == "ram":
        return BitVec("ram", vnode.size*8)
    elif vnode.space.name == "unique":
        return BitVec("unique", vnode.size*8)
    elif vnode.space.name == "register":
        return BitVec(f"register_{vnode.offset}", vnode.size*8)
    else:
        raise ValueError("unrecognized space", vnode.space.name)
def step(op):
    inputs = [varnode(i) for i in op.inputs]
    match op.opcode:
        case OpCode.IMARK:
            return f"IMARK "
        case OpCode.COPY:
            return inputs[0]
        case OpCode.INT_ADD:
            return inputs[0] + inputs[1]
        case OpCode.INT_SUB:
            return inputs[0] - inputs[1]
        case OpCode.STORE:
            pass
        case _:
            print("unrecognized opcode", op.opcode)
            assert False
for op in res.ops:
    print(step(op))

    print(PcodePrettyPrinter.fmt_op(op))

IMARK 
IMARK ram[0:4]
IMARK 
IMARK ram[4:1]
register_40
unique[10c00:8] = RBP
register_32 - 8
RSP = RSP - 0x8
None
*[ram]RSP = unique[10c00:8]
IMARK 
IMARK ram[5:3]
register_32
RBP = RSP
IMARK 
IMARK ram[8:3]
register_40 + 18446744073709551612
unique[4400:8] = RBP + 0xfffffffffffffffc
register_56
unique[6780:4] = EDI
None
*[ram]unique[4400:8] = unique[6780:4]
IMARK 
IMARK ram[b:3]
register_40 + 18446744073709551608
unique[4400:8] = RBP + 0xfffffffffffffff8
register_48
unique[6780:4] = ESI
None
*[ram]unique[4400:8] = unique[6780:4]
IMARK 
IMARK ram[e:3]
register_40 + 18446744073709551608
unique[4400:8] = RBP + 0xfffffffffffffff8
unrecognized opcode OpCode.LOAD


AssertionError: 

In [80]:
from pwn import asm, context
from pypcode import Context, PcodePrettyPrinter

# Set the pwntools context to RISC-V 32-bit
context.arch = 'riscv32'
context.endian = 'little'

# Define a list of simple RISC-V instructions
instructions = [
    "li t0, 42",        # Load immediate
    "add t1, t0, t0",   # Add t0 + t0 -> t1
    "sub t2, t1, t0",   # Subtract t1 - t0 -> t2
    "lw t3, 0(t0)",     # Load word from memory
    "sw t3, 4(t0)"      # Store word to memory
]
# Assemble the instructions into bytes
assembled_code = b"".join(asm(inst) for inst in instructions)

[!] Could not find system include headers for riscv32-linux
